In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requi

In [2]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'iqothnccd-lung-cancer-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1748489%2F2882784%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240724%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240724T202344Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5d5549ff4d884252ac25a120e930d68231439f92e77bbc785ff519c8aca7a79ae81994ed19a1243e0345313887e551875e28585c9f046def634086cff592860ea9a0eb1b15d0d8cbac21306d72029aed5b770d40044871d2e869739dd6f1e33493c878b37c1b2b734d7bf5e1ed2289d4f20508544f943f515ebb74109f18e7d5abc495f1302521dad4b86fe014c365b5dff2c472b42b46798530134a68b37163393401a28c98206473dd92676057c3fefc14cf4b6db095660a8c9d5bc3ed00c5f4f08acc337c8a9f706f0ed7aba32c91a422887a0adc950e66c49b9895aa71e615690264d3163db9704d73d0d040c25b6e52d35da930096ef28f44af4f09acfb'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 208562220 bytes downloaded
Downloaded and uncompressed: iqothnccd-lung-cancer-dataset
Data source import complete.


In [3]:
import os
import random
import torch
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from transformers import ViTFeatureExtractor, ViTForImageClassification, TrainingArguments, Trainer
from datasets import load_metric, Dataset, DatasetDict
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras import layers
from tqdm import tqdm
import cv2

# Define constants
IMAGE_SIZE = 256
PATCH_SIZE = 16
NUM_CLASSES = 3
NUM_LABELS = ['Bengin cases', 'Malignant cases', 'Normal cases']
SEED = 42
BATCH_SIZE = 32
NUM_EPOCHS = 4
LEARNING_RATE = 2e-4

# Define paths
DATA_DIR = '/kaggle/input/iqothnccd-lung-cancer-dataset/The IQ-OTHNCCD lung cancer dataset/The IQ-OTHNCCD lung cancer dataset/'

# Initialize feature extractor
model_name_or_path = 'google/vit-base-patch16-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [4]:
# Load the dataset
data_train = []
for label_id, sp in enumerate(NUM_LABELS):
    for file in os.listdir(os.path.join(DATA_DIR, sp)):
        data_train.append({'image': os.path.join(DATA_DIR, sp, file), 'label': label_id})

df = pd.DataFrame(data_train)
df = df.sample(frac=1, random_state=SEED).reset_index(drop=True)

# Split into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=SEED)

# Define a function to load and resize images
def load_and_resize(image_path):
    image = Image.open(image_path).convert("RGB")
    return image

# Define a function to process each example
def process_example(example):
    image = load_and_resize(example['image'])
    inputs = feature_extractor(images=image, return_tensors='pt')
    inputs['label'] = example['label']
    return inputs

# Create HuggingFace datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Apply processing
train_dataset = train_dataset.map(process_example, remove_columns=['image'])
val_dataset = val_dataset.map(process_example, remove_columns=['image'])

# Wrap in DatasetDict
datasets = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})


Map:   0%|          | 0/877 [00:00<?, ? examples/s]

Map:   0%|          | 0/220 [00:00<?, ? examples/s]

In [10]:
# Define data collator
def collate_fn(batch):
    pixel_values = [torch.tensor(x['pixel_values']).squeeze() if isinstance(x['pixel_values'], list) else x['pixel_values'].squeeze() for x in batch]
    labels = [x['label'] for x in batch]

    pixel_values = torch.stack(pixel_values)
    labels = torch.tensor(labels)

    # Ensure pixel_values are in the shape [batch_size, num_channels, height, width]
    if pixel_values.ndim == 4 and pixel_values.shape[1] != 3:
        pixel_values = pixel_values.permute(0, 3, 1, 2)

    return {
        'pixel_values': pixel_values,
        'labels': labels
    }

# Define evaluation metric
metric = load_metric("accuracy")

def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

# Load pretrained model
model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=NUM_CLASSES,
    id2label={str(i): c for i, c in enumerate(NUM_LABELS)},
    label2id={c: str(i) for i, c in enumerate(NUM_LABELS)}
)

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./vit-lung-cancer",
    per_device_train_batch_size=BATCH_SIZE,
    evaluation_strategy="steps",
    num_train_epochs=NUM_EPOCHS,
    fp16=True,
    save_steps=100,
    eval_steps=100,
    logging_steps=10,
    learning_rate=LEARNING_RATE,
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=False,
    report_to='tensorboard',
    load_best_model_at_end=True,
)

# Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=datasets['train'],
    eval_dataset=datasets['validation'],
    tokenizer=feature_extractor,
)

train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

metrics = trainer.evaluate(datasets['validation'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
100,0.072200,0.078392,0.986364


***** train metrics *****
  epoch                    =         4.0
  total_flos               = 253174766GF
  train_loss               =      0.3044
  train_runtime            =  0:07:24.69
  train_samples_per_second =       7.888
  train_steps_per_second   =       0.252


***** eval metrics *****
  epoch                   =        4.0
  eval_accuracy           =     0.9864
  eval_loss               =     0.0784
  eval_runtime            = 0:00:29.04
  eval_samples_per_second =      7.575
  eval_steps_per_second   =      0.964


In [13]:
# Print the log history to understand the available keys
for log in trainer.state.log_history:
    print(log)


{'loss': 0.7797, 'grad_norm': 0.9752959609031677, 'learning_rate': 0.00018214285714285714, 'epoch': 0.35714285714285715, 'step': 10}
{'loss': 0.4757, 'grad_norm': 20.003475189208984, 'learning_rate': 0.00016428571428571428, 'epoch': 0.7142857142857143, 'step': 20}
{'loss': 0.555, 'grad_norm': 1.4542673826217651, 'learning_rate': 0.00014642857142857141, 'epoch': 1.0714285714285714, 'step': 30}
{'loss': 0.4568, 'grad_norm': 20.9315242767334, 'learning_rate': 0.00013035714285714286, 'epoch': 1.4285714285714286, 'step': 40}
{'loss': 0.2871, 'grad_norm': 0.6445314884185791, 'learning_rate': 0.00011250000000000001, 'epoch': 1.7857142857142856, 'step': 50}
{'loss': 0.2787, 'grad_norm': 3.1732423305511475, 'learning_rate': 9.464285714285715e-05, 'epoch': 2.142857142857143, 'step': 60}
{'loss': 0.166, 'grad_norm': 0.4864746034145355, 'learning_rate': 7.67857142857143e-05, 'epoch': 2.5, 'step': 70}
{'loss': 0.1673, 'grad_norm': 1.1049270629882812, 'learning_rate': 5.8928571428571435e-05, 'epoch'

In [20]:
model.save_pretrained('./vit-lung-cancer-model')
feature_extractor.save_pretrained('./vit-lung-cancer-model')


['./vit-lung-cancer-model/preprocessor_config.json']

In [17]:

from transformers import ViTForImageClassification, ViTFeatureExtractor

# Load the model
model = ViTForImageClassification.from_pretrained('./vit-lung-cancer-model')

# Load the feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)

# Example prediction (you need to replace `image_path` with your actual image path)
image = load_and_resize('/content/Bengin_case.jpg')
inputs = feature_extractor(images=image, return_tensors='pt')
predictions = model(**inputs)

print(predictions)


ImageClassifierOutput(loss=None, logits=tensor([[ 1.8556, -1.0630, -0.8043]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [18]:
print(predictions)


ImageClassifierOutput(loss=None, logits=tensor([[ 1.8556, -1.0630, -0.8043]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [19]:
import torch
from transformers import ViTForImageClassification, ViTFeatureExtractor
from PIL import Image
import matplotlib.pyplot as plt

# Define constants
model_dir = './vit-lung-cancer-model'
image_path = 'Bengin_case.jpg'  # Update this to the path of your image
num_labels = ['Bengin cases', 'Malignant cases', 'Normal cases']

# Load the model and feature extractor
model = ViTForImageClassification.from_pretrained(model_dir)
feature_extractor = ViTFeatureExtractor.from_pretrained(model_dir)

# Define a function to load and resize images
def load_and_resize(image_path):
    image = Image.open(image_path).convert("RGB")
    return image

# Load and preprocess the single image
image = load_and_resize(image_path)
inputs = feature_extractor(images=image, return_tensors='pt')

# Run inference
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    preds = torch.argmax(outputs.logits, dim=-1)

# Print the prediction
predicted_label = num_labels[preds.item()]
print(f'The predicted label for the image is: {predicted_label}')

# Display the image
plt.imshow(image)
plt.title(f'Predicted: {predicted_label}')
plt.axis('off')
plt.show()


OSError: ./vit-lung-cancer-model does not appear to have a file named preprocessor_config.json. Checkout 'https://huggingface.co/./vit-lung-cancer-model/tree/main' for available files.